In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

import matplotlib.pyplot as plt
from aicsimageio import AICSImage
from tifffile import tifffile
import cv2
import multipletau
import czifile
from skimage import io

In [2]:
%pdb

Automatic pdb calling has been turned ON


In [3]:
# path = "/mnt/sda1/Sophie/2-KineticAnalysisData/2-Datas/02-Long-Movie/SnailShadow/"
# files = [
#     "Snail S 061222 E1_2",
#     "Snail S 061222 E2_1",
#     "Snail S 061222 E2_3", 
#     "Snail S 061222 E3_2", 
#     "Snail S 061222 E3_3", ]


path = "/mnt/sda1/Sophie/2-KineticAnalysisData/2-Datas/02-Long-Movie/SnailCoreTATA/"
files = [
#         "Snail CT 061222 E1_1",
#         "Snail CT 061222 E1_2",
#         "Snail CT 061222 E1_3",
#         "Snail CT 061222 E2_1",
        'Snail CT 071222 E2_1',
        'Snail CT 071222 E2_2',
        "Snail CT 071222 E3_1",
        "Snail CT 071222 E3_3",
        ]

track_ext = "_track.csv"
czi_ext = ".czi"

DELTA_T = 0.39
X_SIZE = 0.0920714
Z_SIZE = 0.5
def func_(x, T, c):
    return (( ((T-x)/(c*(T**2))) * np.heaviside((T-x),0.5)) )

In [4]:
for f in files:
    print(f)
    # load image
    img = czifile.imread(path+'/'+f+czi_ext)
    img = AICSImage(path+'/'+f+czi_ext)
    first_channel_data = img.get_image_data("TZYX", C=0, S=0,)
    
    
    #load csv
    track_file = pd.read_csv(path+'/'+f+track_ext)
    track_file.drop(index=[0,1,2], inplace=True)
    track_file['FRAME'] = pd.to_numeric(track_file["FRAME"])
    track_file['POSITION_X'] = pd.to_numeric(track_file["POSITION_X"])
    track_file['POSITION_Y'] = pd.to_numeric(track_file["POSITION_Y"])
    track_file['TRACK_ID'] = pd.to_numeric(track_file["TRACK_ID"])
    track_file['MEAN_INTENSITY_CH1'] = pd.to_numeric(track_file["MEAN_INTENSITY_CH1"])
    track_file['POSITION_T'] = pd.to_numeric(track_file["POSITION_T"])
    track_file['POSITION_Z'] = pd.to_numeric(track_file["POSITION_Z"])
    
    for id_ in np.unique(track_file['TRACK_ID']):

        x = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['POSITION_T'].values-min(track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['POSITION_T'].values)
        y = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['MEAN_INTENSITY_CH1'].values-np.min(track_file[track_file.TRACK_ID==id_].sort_values('FRAME')['MEAN_INTENSITY_CH1'].values)
       
        
        
        mm = int(len(track_file[track_file.TRACK_ID==id_])/2-1)
        if (mm%2)==0: 
            autocor = multipletau.autocorrelate(y,
                                            m=mm,
                                            deltat=DELTA_T, 
                                            normalize=True)
        else: 
            autocor = multipletau.autocorrelate(y,
                                            m=mm+1,
                                            deltat=DELTA_T,
                                            normalize=True)



        popt, pcov = optimize.curve_fit(func_, 
                                    autocor.flatten()[0::2],
                                    autocor.flatten()[1::2], 
                                    method='lm' )
        if popt[0]>5:
            track = track_file[track_file.TRACK_ID==id_].sort_values('FRAME')
            for id_, val in track.iterrows():
                
                
                # Center coordinates
                center_coordinates = (int(val['POSITION_X']/X_SIZE), int(val['POSITION_Y']/X_SIZE))

                # Radius of circle
                radius = 2
                # Line thickness of 2 px
                thickness = 1
                image = cv2.circle(first_channel_data[val['FRAME']][int(val['POSITION_Z']/Z_SIZE)],
                                   center_coordinates,
                                   radius,
                                   np.max(track['MEAN_INTENSITY_CH1'])+5000,
                                   thickness)
    print("before tiff save")
    tifffile.imwrite(path+'/'+f+'_track.tiff',
                     first_channel_data.astype("uint16"),
                     imagej=True,
                     resolution=(1/X_SIZE, 1/X_SIZE),
                     metadata={'spacing': Z_SIZE,
                               'unit': 'um',
                               'axes': 'TZYX',
                               'hyperstack':True,
                               'channels':1,
                               'slices':first_channel_data.shape[1]})
    print('after tiff save')

Snail CT 071222 E2_1
before tiff save
after tiff save
Snail CT 071222 E2_2
before tiff save
after tiff save
Snail CT 071222 E3_1
before tiff save
after tiff save
Snail CT 071222 E3_3
before tiff save
after tiff save


# Crop around track

In [ ]:
import numpy as np
import pandas as pd
from czifile import imread, CziFile

import tifffile as tiff
import matplotlib.pyplot as plt

import napari

In [ ]:
# read one file
path = "/mnt/sda1/Sophie/2-KineticAnalysisData/2-Datas/01-Short_Movie_SnailSuntag/"
files = [
    "1-sna-suntag-fast-imaging/220822 scFv msGFP2 x Snail Core 32X Suntag 24X MS2 E1 z8 t3_Airyscan Processing-20",
    "1-sna-suntag-fast-imaging/220822 scFv msGFP2 x Snail Core 32X Suntag 24X MS2 E1 z8 t4_Airyscan Processing-20",
    "1-sna-suntag-fast-imaging/220822 scFv msGFP2 x Snail Core 32X Suntag 24X MS2 E1 z8 t5_Airyscan Processing-20",
    "2-120822-scfv-msgfp2-x-snail-core-32x-suntag-24-xms2-q670-cy3-flap-y-dapi-e3/120822 ScFv msGFP2 x Snail Core 32X Suntag 24 XMS2 Q670 Cy3 Flap Y DAPI E3 t2 z8_Airyscan Processing-01",
    "3-100822-scfv-msgfp2-x-snail-shadow-32x-suntag-24x-ms2-e2-zoom-8/100822 scFv msGFP2 x Snail Shadow 32X Suntag 24X MS2 E2 zoom 8_Airyscan Processing",
    "3-100822-scfv-msgfp2-x-snail-shadow-32x-suntag-24x-ms2-e2-zoom-8/scFv msGFP2 x Snail Shadow 32X Sunatg 24X MS2 E1 zoom 8_Airyscan Processing-03"]

x_size = 0.0920714
y_size = 0.0920714
z_size = 0.5
dt = 0.39

In [ ]:
im_czi = CziFile(path+files[0]+"/"+files[0].split('/')[-1]+'.czi')
im = imread(path+files[0]+"/"+files[0].split('/')[-1]+'.czi')
f = path+files[0]+"/track_spot_table.csv"
datas = pd.read_csv(f)
datas.drop(index=[0,1,2], inplace=True)
datas['FRAME'] = pd.to_numeric(datas["FRAME"])
datas['POSITION_X'] = pd.to_numeric(datas["POSITION_X"])
datas['POSITION_Y'] = pd.to_numeric(datas["POSITION_Y"])
datas['POSITION_Z'] = pd.to_numeric(datas["POSITION_Z"])
datas['TRACK_ID'] = pd.to_numeric(datas["TRACK_ID"])
datas['MEAN_INTENSITY_CH1'] = pd.to_numeric(datas["MEAN_INTENSITY_CH1"])
datas['POSITION_T'] = pd.to_numeric(datas["POSITION_T"])

# viewer = napari.Viewer(ndisplay=3)
# viewer.add_image(im, scale=[1,1,3])

In [ ]:
tracks = np.unique(datas['TRACK_ID'])
for t in tracks:
    pos_x = np.round((np.min(datas[datas['TRACK_ID']==t]['POSITION_X'])/x_size, np.max(datas[datas['TRACK_ID']==t]['POSITION_X']/x_size)))
    pos_y = np.round((np.min(datas[datas['TRACK_ID']==t]['POSITION_Y'])/y_size, np.max(datas[datas['TRACK_ID']==t]['POSITION_Y']/y_size)))
    pos_z = np.round((np.min(datas[datas['TRACK_ID']==t]['POSITION_Z'])/z_size, np.max(datas[datas['TRACK_ID']==t]['POSITION_Z']/z_size)))
    pos_t = np.round((np.min(datas[datas['TRACK_ID']==t]['POSITION_T'])/z_size, np.max(datas[datas['TRACK_ID']==t]['POSITION_T']/z_size)))
    
        
    im_crop = im[int(pos_t[0]):int(pos_t[1]),
                   0, 
                   int(pos_z[0]):int(pos_z[1]), 
                   int(pos_y[0]):int(pos_y[1]), 
                   int(pos_x[0]):int(pos_x[1])][:,:,:,:,0]
    
    print(t, im_crop.shape)
    tiff.imwrite(path+files[0]+"/Crops/crop_"+str(t)+".tif", 
                 im_crop,
                imagej=True,
                resolution=(1./x_size, 1./y_size),
                metadata={
                'spacing': 0.5,
                'unit': 'um',
                'finterval': dt,
                'axes': 'TZYX'
             })

In [ ]:
im_crop.shape

In [ ]:
im_crop[:,:,:,:,0].shape